In [134]:
import xml.etree.ElementTree as ET
import numpy as np
from copy import deepcopy
#import warnings

In [15]:
tree = ET.parse('../Składnica-frazowa-171220/NKJP_1M_0401000002/morph_2-p/morph_2.52-s.xml')
tree

In [195]:
tree = ET.parse('../Składnica-frazowa-171220/NKJP_1M_0401000002/morph_4-p/morph_4.67-s.xml')
tree

In [245]:
def check_sentence(xml_tree, accept_tags=["forest","tree"]):
    
    """
    Funkcja sprawdza poprawnosc wypowiedzenia i arumentu: 
    - czy istnieje dla niego poprawne drzewo - wypowiedzenie jest poprawne jesli base_answer na polu "type" ma wartosc "FULL".
    - arumentem powinno byc drzewo o tagu korzenia rownym "forest" lub "tree".
    [W oryginalnych plikach z lasami jest to "forest", natomiast gdy z lasu tworzone sa pojedyncze drzewa,
    to maja one tag "tree"]
    
    xml_tree - las drzew lub drzewo [xml.etree.ElementTree.ElementTree]
    """
    
    assert type(xml_tree)== ET.ElementTree, "Bad argument type"
    
    
    if type(accept_tags) == str:
        accept_tags = [accept_tags]
    
    
    if not xml_tree.getroot().tag in accept_tags:
        raise AssertionError('Argument in not in [' + ",".join(accept_tags) + '] - it has tag "' + xml_tree.getroot().tag + '"' )
    
    
    base_answer_type = xml_tree.getroot().find('.//answer-data//base-answer').attrib["type"]
    correct = base_answer_type == "FULL"

    if not correct:
        raise AssertionError("Sentence is not correct: Node <base-answer> has type value " + base_answer_type  + " instead of 'FULL'")
        
    pass


def get_random_tree(forest, random_state=None):
    
    """
    Funkcja zwraca losowe drzewo z upakowanego lasu (forest).
    Dla lasu, w ktorym nie ma poprawnego drzewa funkcja wyrzuca blad.
    
    forest - las drzew [xml.etree.ElementTree.ElementTree]
    """

    # sprawdzenie poprawnosci lasu i ewentualne wypisanie komunikatu
    check_sentence(forest,"forest")
    
    # ustawiamy ziarno
    if random_state is not None:
        np.random.seed(random_state)
            
            
    root_old = forest.getroot()
    root_new = ET.Element("tree",root_old.attrib)
    
    
    # las sklada sie z drzew (wezly "node") oraz dodatkowych danych (inne wezly) -
    # tresc wypowiedzenia, statystyki lasu, itd. - i tutaj przepisujemy te wezly
    features = root_old.getchildren()
    for feature in features:
        if feature.tag != "node": 
            feature_copy = deepcopy(feature)
            if feature_copy.tag == "stats":
                feature_copy.tag = "forest-stats"
                
            root_new.append(feature_copy) # modyfikujemy tag wezla wiec potrzebna kopia, zeby nie zmodyfikowac oryginalnego drzewa
    
    # definiujemy wezel ze statystykami drzewa
    # robimy to w tym iejscu zeby zachowac logiczna kolejnosc wezlow - zeby wypisywalo sie to na poczatku
    # wartosci nadamy nizej
    ET.SubElement(root_new, "tree-stats", {"height":"0","nodes":"0"})
            
            
    # definiujemy rekurencyjna funkcje, ktora bedzie przechodzic po lesie i
    # kolekcjonowac wezly, tworzac losowe drzewo.
    # drzewo jest tworzone na korzeniu root_new.
    def add_random_children(current_node_old):
        
        current_node_new = ET.SubElement(root_new, current_node_old.tag, current_node_old.attrib)
        
        features = current_node_old.getchildren()
        # kazdy "node" jest terminalem albo nieterminalem i ma opis wlasnosci
        # i tutaj wyciagamy te wlasnosci z wezla innego niz "children"
        for feature in features:
            if feature.tag != "children": 
                current_node_new.append(feature)
        
        children_old = current_node_old.findall("children")
        if len(children_old) == 0: #jestesmy w lisciu wiec konczymy dzialanie funkcji
            return None
        random_children_old = children_old[np.random.choice(len(children_old),1)[0]]
        random_children_new = ET.SubElement(current_node_new, random_children_old.tag, random_children_old.attrib)
        for child_old in random_children_old.getchildren():
            x = ET.SubElement(random_children_new, child_old.tag, child_old.attrib)
            next_node = root_old.find('.//node[@nid="' + x.attrib["nid"] + '"]')
            add_random_children(next_node)
        
    
        # wezel startowy (przyjmujemy, ze node z id=0 jest zawsze pierwszy):
    # TODO: upewnic sie czy to jest poprawne podejscie - czy moze byc inny wezel poczatkowym
    node_0 = root_old.find('.//node[@nid="0"]') 
    
    # konstruujemy drzewo:
    add_random_children(node_0)
    
    new_tree = ET.ElementTree(root_new)
    
    th = _tree_height(new_tree, node_id=0)
    
    root_new.find("tree-stats").attrib["height"] = str(th)
    root_new.find("tree-stats").attrib["nodes"] = str(len(root_new.findall("node")))
    
    return new_tree
       
 

def number_of_trees_in_forest(forest):

    """
    Funkcja zwraca liczbe drzew w lesie forest.
    
    forest - las drzew [xml.etree.ElementTree.ElementTree]
    """
    
    check_sentence(forest,"forest")
    
    return int(forest.find("stats").attrib["trees"])
    
    
def get_random_negative_tree(forest):
    
    """
    Funkcja zwraca losowe negatywne (niepoprawne) drzewo z lasy forest.
    
    Gdy las sklada sie tylko z jednego drzewa (poprawnego) to zwracana jest wartosc None.
    
    forest - las drzew [xml.etree.ElementTree.ElementTree]
    """
    
    check_sentence(forest,"forest")
    
    number_of_trees_in_forest = forest.find("stats").attrib["trees"]
    
    if number_of_trees_in_forest == 1:
        Warning("There is only one tree in the forest")
        return None
    
    else:
        while True:
            tree = get_random_tree(forest)
            if not is_positive(tree):
                return tree
    
    

In [246]:
random_tree = get_random_tree(tree)

In [247]:
ET.dump(random_tree)

<tree grammar_no="1505562921" sent_id="NKJP_1M_0401000002/morph_4-p/morph_4.67-s"><text>Papież Formosus zmarł w kwietniu 896 w wyniku nieznanej choroby lub otrucia.</text>
  <startnode from="0" to="13">wypowiedzenie</startnode>
  <forest-stats cputime="10.487813580000001" inferences="7448127" nodes="124" trees="173" />
    <answer-data>
        <base-answer type="FULL" username="none">
            <comment>AUTO</comment>
        </base-answer>
        <extra-answer type="FULL" username="witoldk">
            <comment>AUTO</comment>
        </extra-answer>
        <extra-answer type="FULL" username="piotrb">
            <comment>AUTO</comment>
        </extra-answer>
    </answer-data>
  <tree-stats height="15" nodes="49" /><node chosen="true" from="0" nid="0" subtrees="173" to="13"><nonterminal>
      <category>wypowiedzenie</category>
    </nonterminal>
    <children chosen="true" rule="w"><child from="0" head="true" nid="1" to="12" /><child from="12" head="false" nid="71" to="13" /><

In [184]:
random_tree.write("test.xml")

TypeError: cannot serialize 15 (type int)

In [233]:
ET.dump(random_tree)

<tree grammar_no="1505562921" sent_id="NKJP_1M_0401000002/morph_4-p/morph_4.67-s"><text>Papież Formosus zmarł w kwietniu 896 w wyniku nieznanej choroby lub otrucia.</text>
  <startnode from="0" to="13">wypowiedzenie</startnode>
  <forest-statsxxxxxxxxxxxx cputime="10.487813580000001" inferences="7448127" nodes="124" trees="173" />
    <answer-data>
        <base-answer type="FULL" username="none">
            <comment>AUTO</comment>
        </base-answer>
        <extra-answer type="FULL" username="witoldk">
            <comment>AUTO</comment>
        </extra-answer>
        <extra-answer type="FULL" username="piotrb">
            <comment>AUTO</comment>
        </extra-answer>
    </answer-data>
  <node chosen="true" from="0" nid="0" subtrees="173" to="13"><nonterminal>
      <category>wypowiedzenie</category>
    </nonterminal>
    <children chosen="true" rule="w"><child from="0" head="true" nid="1" to="12" /><child from="12" head="false" nid="71" to="13" /></children></node><node ch

In [121]:
tree = random_tree

In [213]:
t

In [212]:
random_tree = t.getroot()

In [144]:
type(tree)

xml.etree.ElementTree.ElementTree

In [172]:
def _tree_height(xml_tree, node_id=0):
    
    """
    Funkcja oblicza wysokosc drzewa (dlugosc najdluzszej sciezki od korzenia do liscia)
    lub lasu (maximum z wszystkich mozliwych drzew)
    
    xml_tree - drzewo luba las drzew lub korzen drzewa jednego lub drugiego
    """
    
    
    if type(xml_tree)==ET.Element:
        node = tree
    else:
        node = tree.getroot()
        
    node = node.find('.//node[@nid="' + str(node_id) + '"]')
    children = node.findall(".//children//child")
    
    if len(children)==0:
        return 1
    else:
        children_nodes = [child.attrib["nid"] for child in children]
        return 1+max([_tree_height(tree,x) for x in children_nodes])
        

In [173]:
_tree_height(tree.getroot())

15

In [205]:
def number_of_nodes(tree):
    """
    Zwraca liczbe wezlow w drzewie.
    
    tree - drzewo lub korzen drzewa
    """
    if type(tree)==ET.Element:
        return len(tree.findall("node"))
    else:
        return len(tree.getroot().findall("node")) 

In [215]:
number_of_nodes(t)

49

In [151]:
ET.dump(t)

<forest grammar_no="1505562921" sent_id="NKJP_1M_0401000002/morph_4-p/morph_4.67-s"><text>Papież Formosus zmarł w kwietniu 896 w wyniku nieznanej choroby lub otrucia.</text>
  <startnode from="0" to="13">wypowiedzenie</startnode>
  <forest_stats cputime="10.487813580000001" inferences="7448127" nodes="124" trees="173" />
    <answer-data>
        <base-answer type="FULL" username="none">
            <comment>AUTO</comment>
        </base-answer>
        <extra-answer type="FULL" username="witoldk">
            <comment>AUTO</comment>
        </extra-answer>
        <extra-answer type="FULL" username="piotrb">
            <comment>AUTO</comment>
        </extra-answer>
    </answer-data>
  <node chosen="true" from="0" nid="0" subtrees="173" to="13"><nonterminal>
      <category>wypowiedzenie</category>
    </nonterminal>
    <children chosen="true" rule="w"><child from="0" head="true" nid="1" to="12" /><child from="12" head="false" nid="71" to="13" /></children></node><node chosen="true

In [149]:
random_tree

<Element 'forest' at 0x7f66dae6e408>

In [150]:
tree_height(random_tree)

4

In [81]:
def is_positive(tree): 
    
    """
    Funkcja sprawdza czy drzewo jest pozytywne - czy jest poprawnym drzewem rozbioru
    Zwraca wartosc logiczna.
    
    tree - drzewo [xml.etree.ElementTree.ElementTree]
    """
    
    check_sentence(tree,"tree")
    
    assert len(tree.find("node"))>0, 'There is not "node" element in the tree'
    
    #Sprawdzamy czy wszystkie wezly "node" maja wartosc chosen="true":
    for x in tree.iter("node"):
        if not x.attrib["chosen"]=="true":
            return False

    #Sprawdzamy czy wszystkie wezly "node" maja wartosc chosen="true":
    for x in tree.iter(".//children"):
        if not x.attrib["chosen"]=="true":
            return False
        
    return True
        
        
    


def get_positive_tree(forest):
    
    """
    Funkcja zwraca poprawne (pozytywne) drzewo z upakowanego lasu (forest).
    Dla lasu, w ktorym nie ma poprawnego drzewa funkcja wyrzuca blad.
    
    forest - las drzew [xml.etree.ElementTree.ElementTree]
    """

    # sprawdzenie poprawnosci lasu i ewentualne wypisanie komunikatu
    check_sentence(forest,"forest")
            
    root_old = forest.getroot()
    root_new = ET.Element("tree",root_old.attrib)
    
    
    # las sklada sie z drzew (wezly "node") oraz dodatkowych danych (inne wezly) -
    # tresc wypowiedzenia, statystyki lasu, itd. - i tutaj przepisujemy te wezly
    features = root_old.getchildren()
    for feature in features:
        if feature.tag != "node": 
            feature_copy = deepcopy(feature)
            if feature_copy.tag == "stats":
                feature_copy.tag = "forest-stats"
                
            root_new.append(feature_copy) # modyfikujemy tag wezla wiec potrzebna kopia, zeby nie zmodyfikowac oryginalnego drzewa
            
    # definiujemy rekurencyjna funkcje, ktora bedzie przechodzic po lesie i
    # kolekcjonowac wezly, tworzac losowe drzewo.
    # drzewo jest tworzone na korzeniu root_new.
    def add_positive_children(current_node_old):
        
        current_node_new = ET.SubElement(root_new, current_node_old.tag, current_node_old.attrib)
        
        features = current_node_old.getchildren()
        # kazdy "node" jest terminalem albo nieterminalem i ma opis wlasnosci
        # i tutaj wyciagamy te wlasnosci z wezla innego niz "children"
        for feature in features:
            if feature.tag != "children": 
                current_node_new.append(feature)
        
        
        children_old = current_node_old.findall('children[@chosen="true"]')
        # powinno byc tylko jedno takie dziecko
        
        assert len(children_old) <= 1, 'More than one children has chosen="true"'
        
        if len(children_old) == 0: #jestesmy w lisciu wiec konczymy dzialanie funkcji
            return None
        
        #random_children_old = children_old[np.random.choice(len(children_old),1)[0]]
        children_new = ET.SubElement(current_node_new, children_old[0].tag, children_old[0].attrib)
        for child_old in children_old[0].getchildren():
            x = ET.SubElement(children_new, child_old.tag, child_old.attrib)
            next_node = root_old.find('.//node[@nid="' + x.attrib["nid"] + '"]')
            assert next_node.attrib["chosen"] == "true"
            add_positive_children(next_node)
        
    
    # wezel startowy (przyjmujemy, ze node z id=0 jest zawsze pierwszy):
    # TODO: upewnic sie czy to jest poprawne podejscie - czy moze byc inny wezel poczatkowym
    node_0 = root_old.find('.//node[@nid="0"][@chosen="true"]') 
    
    # konstruujemy drzewo:
    add_positive_children(node_0)
    
    positive_tree = ET.ElementTree(root_new)

    # Sprawdzenie poprawnosci drzewa
    assert is_positive(positive_tree), """Something gone wrong - tree is not positive"""
        
        
    return positive_tree
    

In [82]:
pos_tree = get_positive_tree(tree)

In [83]:
get_positive_tree(tree).write("test.xml")

In [87]:
pos_tree.findall("node")

[<Element 'node' at 0x7f6ded401f98>,
 <Element 'node' at 0x7f6ded401868>,
 <Element 'node' at 0x7f6ded4015e8>,
 <Element 'node' at 0x7f6ded401cc8>,
 <Element 'node' at 0x7f6ded401b88>,
 <Element 'node' at 0x7f6ded401778>,
 <Element 'node' at 0x7f6ded3e5048>,
 <Element 'node' at 0x7f6ded3e50e8>,
 <Element 'node' at 0x7f6ded3e51d8>,
 <Element 'node' at 0x7f6ded3e52c8>,
 <Element 'node' at 0x7f6ded3e5368>,
 <Element 'node' at 0x7f6ded3e5458>,
 <Element 'node' at 0x7f6ded3e5548>,
 <Element 'node' at 0x7f6ded4017c8>,
 <Element 'node' at 0x7f6ded401ea8>,
 <Element 'node' at 0x7f6ded401d68>,
 <Element 'node' at 0x7f6ded401ef8>,
 <Element 'node' at 0x7f6ded3ef9f8>,
 <Element 'node' at 0x7f6ded3eff98>,
 <Element 'node' at 0x7f6ded3efa98>,
 <Element 'node' at 0x7f6ded3efbd8>,
 <Element 'node' at 0x7f6ded3e5638>,
 <Element 'node' at 0x7f6ded3e56d8>,
 <Element 'node' at 0x7f6ded3e57c8>,
 <Element 'node' at 0x7f6ded3e58b8>,
 <Element 'node' at 0x7f6ded3e59a8>,
 <Element 'node' at 0x7f6ded3e5a48>,
 